# Задача классификации тональности текста

In [0]:
!pip install deeppavlov
!pip install -U scikit-learn

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import json

from __future__ import division
from collections import Counter
import numba as nb
import re, nltk

import os

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Загрузка данных
Если вы используете Google Colab, предварительно загрузите файл dataset_sentiment.json в файлы доступные ноутбуку.

In [0]:
with open('dataset_sentiment.json') as f:
    raw_data = json.load(f)

In [0]:
texts, sents = [raw['input'] for raw in raw_data], [raw['payload'].split('_')[0] for raw in raw_data]
raw_df = pd.DataFrame.from_dict({'text': texts, 'sentiment': sents})

In [6]:
pd.set_option('display.max_colwidth', -1)
raw_df.head(10)

,sentiment,text
0,neutral,Предлагаем Вашему вниманию туры в ОАЭ с вылетом 3 мая Вылет: из Санкт-Петербурга Перелет: АК Россия (группа Аэрофлот) Продолжительность: на 3 ночи Цена от 18302 руб!!!
1,positive,"Valter Lekhman shared a memory. Valter В 2012 г, когда я переезжал на Бали , в день моего отъезда в журнале ""Аэрофлот"" вышла статья с таким вот заголовком )) думаю это знак и у меня здесь все получится!!!"
2,positive,А я новую ночь превращаю в рассвет
3,neutral,"Предлагаем вашему вниманию туры в Прагу! Перелет: АК Россия (группа Аэрофлот) Гарантированные номера в лучших отелях 3-5*… 23.03 продолжительность от 3 ночей, цена от 22 068 руб. Спешите бронировать! Еще"
4,positive,Очередная командировка в Китай🇨🇳 Из Москвы SVO летим в Гуанчжоу CAN. Как всегда с любимым ️ ялюблютебя моимиглазами фото моя грам 2018 neversleep walks now today
5,negative,Игорь Зинченко shared a link. Л. Якубович: Аэрофлот - мразь!youtube.com15 марта 2013 года в Шереметьево был задержан рейс Аэрофлота SU-292 Москва-Хошимин. Время вылета переносилось несколько раз без всяких объяснений. После очер...
6,neutral,Когда его нет рядом- я на земле. Ольга Елохина added a new photo — at Тренажерный Комплекс Аэрофлот. Instagram Photos
7,neutral,"Обещал и сделалдля тех кто пропустил! Собирал модель самолета и говорил что следующий на очереди и вот ОН! Красавец, в цвете Модель компании краски"
8,neutral,"Может кто-то передумал лететь в Москву))) на Аэрофлот, куплю билет за приемлемо ) срочно на рейсы этих выходных_10/03_либо 11/03 _1взр.спасибо."
9,neutral,"шо опять вы там что... решили хором загнуться🤔 В Лондоне умер Николай Глушков. «Друг Березовского». Ему было 68 лет. В 2017 в России был приговорен к 8 годам по делу Аэрофлота. В 2010 году получил политическое убежище в Британии. Стесняюсь спросить, мы уже виноваты или Лондон ещё думает?"


## Препроцессинг текста

Глядя на данные что мы имеем можно сделать вывод, что обработка текста будет делом не из простых.
Вот список задач, которые мы должны решить, чтобы успешно токенизировать наши данные:
1. Удалить английские слова. Глянув на содержание этих слов, можно понять, что это по большей части программные вставки, подобно "added a new photo". Эти слова, на мой взгляд будут только мешать.
2. Очистить текст от пунктуаций. Хотя здесь немного спорный момент: некоторые токенайзеры, к примеру, обращают внимание на количество знаков восклицания. От себя могу добавить, что наличие знаков ")" можно было бы использовать как метрику при определении положительной тональности. Однако, моя попытка применить данную идею не увенчалась успехом. Либо это не работает, либо я подготовил данные коряво. (скорее всего верно второе)
3. Убрать числа. Данный вид данных не имеет практической пользы для текущей задачи классификации.
4. Избавиться от имоджи. Хоть некоторые виды имоджи могут быть полезны при определении тональности, многие не несут в себе никакого смысла.
5. Перевести слова в нижний регистр.
6. Разделить слипшиеся слова. На мой вгляд, данный пункт очень важен. Смотря на выборку текстов сверху, можно заметить примеры слипшихся слов, как например "ялюблютебя" и "сделалдля". Удалять такие слова было бы нецелесообразно, так как неизвестно сколько таких слов может быть и какую важность они могут нести.
7. Лемматезировать слова. Это необходимо для сокращения словаря данных.


Используем регулярные выражения для фильтрации имоджи

In [0]:
# избавляемся от иможди
emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)

def remove_emoji(string):
    return emoji_pattern.sub(r'', string)

Выполним пункту с 1 по 5 (см. выше).

In [0]:
def no_emoji_tokenize(string):
  words = word_tokenize(string)
  return map(remove_emoji, words)
  
def ru_token(string):
    """russian tokenize based on nltk.word_tokenize. only russian letter remaind."""
    return [i.lower() for i in no_emoji_tokenize(string) if re.match(r'[\u0400-\u04ffа́]+$', i)]

In [0]:
data_df = raw_df.copy()
data_df['text'] = raw_df['text'].apply(ru_token)

Посмотрим на текущий вид данных

In [10]:
data_df.head(20)

,sentiment,text
0,neutral,"[предлагаем, вашему, вниманию, туры, в, оаэ, с, вылетом, мая, вылет, из, перелет, ак, россия, группа, аэрофлот, продолжительность, на, ночи, цена, от, руб]"
1,positive,"[в, г, когда, я, переезжал, на, бали, в, день, моего, отъезда, в, журнале, аэрофлот, вышла, статья, с, таким, вот, заголовком, думаю, это, знак, и, у, меня, здесь, все, получится]"
2,positive,"[а, я, новую, ночь, превращаю, в, рассвет]"
3,neutral,"[предлагаем, вашему, вниманию, туры, в, прагу, перелет, ак, россия, группа, аэрофлот, гарантированные, номера, в, лучших, отелях, продолжительность, от, ночей, цена, от, руб, спешите, бронировать, еще]"
4,positive,"[очередная, командировка, в, китай, из, москвы, летим, в, гуанчжоу, как, всегда, с, любимым, ялюблютебя, моимиглазами, фото, моя, грам]"
5,negative,"[игорь, зинченко, якубович, аэрофлот, мразь, марта, года, в, шереметьево, был, задержан, рейс, аэрофлота, время, вылета, переносилось, несколько, раз, без, всяких, объяснений, после, очер]"
6,neutral,"[когда, его, нет, я, на, земле, ольга, елохина, тренажерный, комплекс, аэрофлот]"
7,neutral,"[обещал, и, сделалдля, тех, кто, пропустил, собирал, модель, самолета, и, говорил, что, следующий, на, очереди, и, вот, он, красавец, в, цвете, модель, компании, краски]"
8,neutral,"[может, передумал, лететь, в, москву, на, аэрофлот, куплю, билет, за, приемлемо, срочно, на, рейсы, этих]"
9,neutral,"[шо, опять, вы, там, что, решили, хором, загнуться, в, лондоне, умер, николай, глушков, друг, березовского, ему, было, лет, в, в, россии, был, приговорен, к, годам, по, делу, аэрофлота, в, году, получил, политическое, убежище, в, британии, стесняюсь, спросить, мы, уже, виноваты, или, лондон, ещё, думает]"


### Займемся разделение слипшихся слов.
Я не нашел готового библиотек для решения данной задачи. Пришлось искать примеры разделения слипшихся слов на английском и модифицировать их под свои нужды.
По большей части я использовал данные ресурсы:
- Сайт Национальный корпуса русского языка: http://ruscorpora.ru/index.html
- Word Segmentation by Peter Norwig: https://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb#(5)-Task:-Word-Segmentation

Идея

Мы будем использовать частотный словарь русского корпуса.
Будем проходить по словам в данных. Если находим слово, которое не существует в словаре, будем делить его на части полным перебором. При каждом разделении вычисляем выроятность того, что данное разделение возможно, основываясь на частотных вероятностях полученных частей слова. В итоге выбираем разделение наиболее вероятное.

Для ускорения процесса будем кешировать данные.

Скачиваем корпус русского языка

In [11]:
if not os.path.isfile('./corpus/1grams-3.txt'):
  if not os.path.exists('./corpus'):
    os.makedirs('./corpus')
  print("Скачивание архива с файлом, содержащим частоты словоформ.")
  import urllib.request
  import zipfile
  url = 'http://www.ruscorpora.ru/ngrams/1grams-3.zip'  
  urllib.request.urlretrieve(url, './corpus/1grams-3.zip')
  with zipfile.ZipFile('./corpus/1grams-3.zip','r') as zip_ref:
    zip_ref.extractall('./corpus/')
  os.remove('./corpus/1grams-3.zip')
  print("Скачивание окончено.")
  

Скачивание архива с файлом, содержащим частоты словоформ.
Скачивание окончено.


In [0]:
def get_corpus_dict():
  dic = dict()
  with open('corpus/1grams-3.txt') as f:
    for line in f.readlines():
      freq, word = line.strip().split("\t")
      dic[word] = int(freq)

  return dic

In [13]:
corpus_dict = get_corpus_dict()
print("Количество слов в корпусе:", len(corpus_dict))

Количество слов в корпусе: 1054210


In [0]:
def print_dict(dic, n=5):
  i = 0
  print("word -- key")
  for k, v in dic.items():
    if i == n:
      break
    print(k, "--", v)
    i += 1

In [15]:
print_dict(corpus_dict)

word -- key
и -- 6829968
в -- 5190215
не -- 3164900
на -- 2729210
с -- 2064408


Как вы можем видеть сверху, у каждого слова есть значение частоты. Однако, нам нужно превести это в вероятностную величину в рамках [0, 1]. 

Однако не все так глядко. Сумма частот оказалась слишком большой, поэтому будем использовать иной метод нормализации частоты для получения вероятности.

Так мы будем нормализовать частоты, деля на максимальное значение в словаре, то есть на значение слова "и" = 6829968

In [16]:
FREQ_MAX = max(corpus_dict.values())
FREQ_MAX

6829968

Функция нормализации

In [0]:
freq_normalize = lambda x: x / FREQ_MAX

Нормализуем данные

In [0]:
corpus_dict = {k: freq_normalize(v) for k, v in corpus_dict.items()}

In [19]:
print_dict(corpus_dict)

word -- key
и -- 1.0
в -- 0.7599179088393972
не -- 0.46338430868197333
на -- 0.39959338023252816
с -- 0.3022573458616497


Используем функцию враппер для кеширования аргументов для ускорения.

In [0]:
def memo(f):
    "Memoize function f, whose args must all be hashable."
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

Функционал для сегметации слов на части на основе частотных вероятностей.

In [0]:
def pdist(word):
    "Make a probability distribution, given evidence from a Counter."
    if word in corpus_dict:
      return corpus_dict[word]
    else:
      return 0

def Pwords(words):
    "Probability of words, assuming each word is independent of others."
    return np.prod([pdist(w) for w in words])

def splits(text, start=0, L=20):
    "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

@memo
def segment(text):
    "Return a list of words that is the most probable segmentation of text."
    if not text: 
        return []
    else:
      candidates = ([first] + segment(rest) 
                    for (first, rest) in splits(text, 1))
      return max(candidates, key=Pwords)

Протестируем сегментацию

In [22]:
%time print(segment('ялюблютебя'))

['я', 'люблю', 'тебя']
CPU times: user 1.65 ms, sys: 0 ns, total: 1.65 ms
Wall time: 1.51 ms


Функция для сегментирования слов в данных. Важный момент: после, стоит очистить данные от слов длиной меньше 3 букв, ибо они врядли будут нести смысл в текст и считаются скорее как stop words. Хоть очистка от stop words будет еще применена в дальнейшем, дополнительная очистка не помешает.

In [0]:
def segment_words(words):
  new_words = []
  for word in words:
    if word in corpus_dict:
      new_words.append(word)
    else:
      new_words.extend(segment(word))
  new_words = list(filter(lambda w: len(w)>2, new_words))
  return new_words

In [0]:
data_df['text'] = data_df['text'].apply(segment_words)

Поглядим на получившиеся данные.

In [25]:
data_df.head(10)

,sentiment,text
0,neutral,"[предлагаем, вашему, вниманию, туры, вылетом, мая, вылет, перелет, россия, группа, аэрофлот, продолжительность, ночи, цена, руб]"
1,positive,"[когда, переезжал, бали, день, моего, отъезда, журнале, аэрофлот, вышла, статья, таким, вот, заголовком, думаю, это, знак, меня, здесь, все, получится]"
2,positive,"[новую, ночь, превращаю, рассвет]"
3,neutral,"[предлагаем, вашему, вниманию, туры, перелет, россия, группа, аэрофлот, гарантированные, номера, лучших, отелях, продолжительность, ночей, цена, руб, спешите, бронировать, еще]"
4,positive,"[очередная, командировка, китай, москвы, летим, гуан, чжоу, как, всегда, любимым, люблю, тебя, моим, глазами, фото, моя, грам]"
5,negative,"[горь, зин, чен, кубов, аэрофлот, мразь, марта, года, шер, меть, был, задержан, рейс, аэрофлота, время, вылета, переносилось, несколько, раз, без, всяких, объяснений, после, чер]"
6,neutral,"[когда, его, нет, земле, тренажерный, комплекс, аэрофлот]"
7,neutral,"[обещал, сделал, для, тех, кто, пропустил, собирал, модель, самолета, говорил, что, следующий, очереди, вот, красавец, цвете, модель, компании, краски]"
8,neutral,"[может, передумал, лететь, аэрофлот, куплю, билет, приемлемо, срочно, рейсы, этих]"
9,neutral,"[опять, там, что, решили, хором, загнуться, умер, николай, ушко, друг, березовского, ему, было, лет, россии, был, приговорен, годам, делу, аэрофлота, году, получил, политическое, убежище, брит, стесняюсь, спросить, уже, виноваты, или, ещё, думает]"


### Лемматизация.

Как уже было сказано, лемматизация важна для сокращения словаря наших данных. Более того, она снижает смысловую нагрузку, упрощая и стандартизируя слова.


Для данной цели, будем использовать токенайзер с лемматизацией русского языка от DeepPavlov.

In [26]:
from deeppavlov.models.tokenizers.ru_tokenizer import RussianTokenizer
rus_tokenizer = RussianTokenizer(lemmas=True, stopwords=stopwords.words('russian'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


Функция полной токенизации, основанной на всем, что мы сделали до этого.

In [0]:
def final_tokenizer(string):
  words = segment_words(ru_token(string))
  return rus_tokenizer([" ".join(words)])[0]

## Создадим TF-IDF векторизатор
Будем использовать векторизированные данные для обучения моделей.

In [0]:
params = {}
params['tokenizer'] = final_tokenizer
params['stop_words'] = stopwords.words('russian')
params['ngram_range'] = (1, 3)
params['min_df'] = 3

In [0]:
tfidf  = TfidfVectorizer(**params)

In [31]:
tfidf.fit([i for i in raw_df['text']])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', '...гда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между'],
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function final_tokenizer at 0x7f7df6a2ee18>,
        use_idf=True, vocabulary=None)

## Разделим данные на обучающие и тестовые выборки.
Разделение будем проводить честно: делим данные отдельно по каждому виду тональности.

In [0]:
train = {}
test = {}
tmp = defaultdict(list)
for _, row in raw_df.iterrows():
    tmp[row['sentiment']].append(row['text'])
for l in tmp:
    train[l], test[l] = train_test_split(tmp[l], test_size=0.2, random_state=2019)

Также, проводим выравнивание количества данных по каждому классу тональности в обучающей выборке.

In [0]:
def upsampling_align(some_dict, random_state=2018):
    rand = np.random.RandomState(random_state)
    upper = max([len(some_dict[l]) for l in some_dict])
    print('upper bound: {}'.format(upper))
    tmp = {}
    for l in some_dict:
        if len(some_dict[l]) < upper:
            repeat_time = int(upper/len(some_dict[l]))
            remainder = upper % len(some_dict[l])
            _tmp = some_dict[l].copy()
            rand.shuffle(_tmp)
            tmp[l] = some_dict[l] * repeat_time + _tmp[:remainder]
            rand.shuffle(tmp[l])
        else:
            tmp[l] = some_dict[l]
    return tmp

In [34]:
train_data = upsampling_align(train)

upper bound: 8808


## Обучение Logistic Regression классификатора с Softmax функцией для мультиклассификации

In [0]:
m_params = {}
m_params['solver'] = 'lbfgs'
m_params['multi_class'] = 'multinomial'

In [0]:
softmax = LogisticRegression(**m_params)

In [0]:
x_train = [j for i in sorted(train_data.keys()) for j in train_data[i]]
y_train = [i for i in sorted(train_data.keys()) for j in train_data[i]]
x_test = [j for i in sorted(test.keys()) for j in test[i]]
y_test = [i for i in sorted(test.keys()) for j in test[i]]

Пропускаем данные через TF-IDF векторизатор

In [0]:
x_train_tr = tfidf.transform(x_train)
x_test_tr = tfidf.transform(x_test)

In [39]:
softmax.fit(x_train_tr, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

### Оценка модели

In [0]:
pred = softmax.predict(x_test_tr)

In [41]:
accuracy_score(y_test, pred)

0.6881081081081081

Подробный репорт по оценкам и количество семплов по каждому классу тональности

In [42]:
lab = LabelEncoder()
c_test = lab.fit_transform(y_test)
c_pred = lab.transform(pred)
print(classification_report(c_test, c_pred, target_names=lab.classes_, digits=5))

              precision    recall  f1-score   support

    negative    0.47368   0.58442   0.52326       539
     neutral    0.80258   0.70527   0.75079      2202
    positive    0.61636   0.70699   0.65857       959

   micro avg    0.68811   0.68811   0.68811      3700
   macro avg    0.63088   0.66556   0.64420      3700
weighted avg    0.70640   0.68811   0.69374      3700



Если судить по тем примерам, которые я встречал в интернете, данный результат весьма хорош для логистической регрессии.

Однако, на этом мои возможности моделирования заканчиваются. От части это связано с невысокими техническими параметрами машины, а отчасти с моим непрофессионализмом в данном вопросе.

Я попытался применить иные стандартные модели из библиотеки sklearn, а также xgboost. Однако, они не показывали существенного прироста в точности (а то и работали хуже).

Также я попытался смоделировать рекурентную нейронную сеть на Keras и обучить ее на векторах созданных при помощи Word2Vec. Результат получился ужасным, модель получалась переобученной, а ее точность даже не достигала результата логистической регрессии.

# Анализ тональности текста с использованием Keras и Word2Vec
# (Опасно! Не открывать!)

In [43]:
import pandas as pd # provide sql-like data manipulation tools. very handy.
pd.options.mode.chained_assignment = None
import numpy as np # high dimensional vector computing library.
from copy import deepcopy
from string import punctuation
from random import shuffle

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [0]:
from deeppavlov.models.tokenizers.ru_tokenizer import RussianTokenizer
rus_tokenizer = RussianTokenizer(lemmas=True, stopwords=stopwords.words('russian'))

In [0]:
x_train_w = rus_tokenizer(x_train)
x_test_w = rus_tokenizer(x_test)

In [47]:
def labelizeTexts(strings, label_type):
    labelized = []
    for i,v in enumerate(strings):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train_l = labelizeTexts(x_train_w, 'TRAIN')
x_test_l = labelizeTexts(x_test_w, 'TEST')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [48]:
x_train_l[0]

LabeledSentence(words=['swiss', 'получать', 'шкала', 'интернет', 'борт', 'рубль', 'давать'], tags=['TRAIN_0'])

In [0]:
n_dim = 200

In [50]:
w2v_model = Word2Vec(size=n_dim, min_count=10)
w2v_model.build_vocab([x.words for x in x_train_l])
w2v_model.train([x.words for x in x_train_l], total_examples=w2v_model.corpus_count, epochs=w2v_model.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(1693910, 2181430)

In [0]:
# w2v_model.most_similar('плохо')  # результаты схожести весьма ужасны: слова никак не связаны по смыслу. Возможно здесь кроится главная проблема, почему модель не работает
                                    # Однако я не нашел способа исправить это

In [52]:
tfidf_vals = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
print('vocab size :', len(tfidf_vals))

vocab size : 25452


In [0]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf_vals[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [54]:
from sklearn.preprocessing import scale
train_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x.words, x_train_l)])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, n_dim) for z in map(lambda x: x.words, x_test_l)])
test_vecs_w2v = scale(test_vecs_w2v)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [55]:
train_vecs_w2v.shape

(26424, 200)

In [0]:
num_classes = 3

In [0]:
def sent_to_int(sent):
  if sent == "positive":
    return 2
  elif sent == "neutral":
    return 1
  elif sent == "negative":
    return 0
  else:
    print(sent)
    raise ValueError()

In [0]:
y_train_int = list(map(sent_to_int, y_train))
y_test_int = list(map(sent_to_int, y_test))

y_train_int = keras.utils.to_categorical(y_train_int, num_classes)
y_test_int = keras.utils.to_categorical(y_test_int, num_classes)

In [59]:
from keras import regularizers

model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_dim, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, y_train_int, epochs=20, batch_size=10, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
 - 5s - loss: 1.2342 - acc: 0.5661
Epoch 2/20
 - 3s - loss: 0.9606 - acc: 0.5733
Epoch 3/20
 - 3s - loss: 0.9548 - acc: 0.5780
Epoch 4/20
 - 3s - loss: 0.9500 - acc: 0.5755
Epoch 5/20
 - 3s - loss: 0.9495 - acc: 0.5768
Epoch 6/20
 - 3s - loss: 0.9443 - acc: 0.5793
Epoch 7/20
 - 3s - loss: 0.9429 - acc: 0.5783
Epoch 8/20
 - 3s - loss: 0.9433 - acc: 0.5791
Epoch 9/20
 - 3s - loss: 0.9419 - acc: 0.5795
Epoch 10/20
 - 3s - loss: 0.9431 - acc: 0.5786
Epoch 11/20
 - 3s - loss: 0.9420 - acc: 0.5808
Epoch 12/20
 - 3s - loss: 0.9413 - acc: 0.5794
Epoch 13/20
 - 3s - loss: 0.9411 - acc: 0.5809
Epoch 14/20
 - 3s - loss: 0.9399 - acc: 0.5781
Epoch 15/20
 - 3s - loss: 0.9392 - acc: 0.5789
Epoch 16/20
 - 3s - loss: 0.9393 - acc: 0.5802
Epoch 17/20
 - 3s - lo

In [60]:
score = model.evaluate(test_vecs_w2v, y_test_int, batch_size=10, verbose=2)
score[1]

0.5437837882823235

## Использованные ресурсы
- Сайт Национальный корпуса русского языка: http://ruscorpora.ru/index.html
- Emoji pattern: https://stackoverflow.com/a/54313085
- Stuck words splitter: https://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb#(5)-Task:-Word-Segmentation
- Keras Documentation: https://keras.io/
- Sentiment analysis on Twitter using word2vec and keras: https://ahmedbesbes.com/sentiment-analysis-on-twitter-using-word2vec-and-keras.html
- Micro-Macro Precision,Recall and F-Score: https://medium.com/@ramit.singh.pahwa/micro-macro-precision-recall-and-f-score-44439de1a044